In [1]:
import pandas as pd
import numpy as np
import cv2
import itertools


from cycling_manager.preprocess import split, get_data, preprocess

from cycling_manager.sequences import get_sequences, get_sequence

from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Input
import tensorflow as tf

In [2]:
df = preprocess(get_data())

full df shape (603619, 25)
full df shape (603619, 25)
Score vertical meters - model: DecisionTreeRegressor() score: 0.9995644759916735
Score vertical meters - model: DecisionTreeRegressor() score: 0.9999281522775233
Score vertical meters - model: KNeighborsClassifier() score: 0.9937832209132449


In [3]:
train, test = split(df, start=2020, end=2021)

In [4]:
df.columns

Index(['name', 'year', 'type', 'date', 'result', 'gc', 'icon', 'race_ref',
       'race_name', 'race_detail', 'race_rank', 'distance', 'points',
       'Avg. speed winner:', 'Distance:', 'Parcours type:', 'ProfileScore:',
       'Vert. meters:', 'Startlist quality score:', 'Won how: ',
       'adjusted_points', 'Race category:', 'Points scale:', 'Arrival:',
       'Won how:', 'parcours_type_num', 'gt_binary', 'key', 'result_bin',
       'types_bin', 'icon_bin'],
      dtype='object')

In [5]:
y_dec_ls = []
tour_ls = []
season_ls = []
result_ls = []

for name, year, tour in train.values:
    y_decoder, tour_data, result, season_data = get_sequence(df, name, year, tour, img=True, binary=False)
    y_dec_ls.append(y_decoder)
    tour_ls.append(tour_data)
    result_ls.append(result)
    season_ls.append(season_data)

In [6]:
season_ls_img = []
to_drop_ls = []

for season in season_ls:
    base_path = '../raw_data/img_300/'
    season = [base_path+"_".join(race.split('/')[1:])+'.jpg' for race in season]
    
    img_ls = []
    
    for race in season:
        img = cv2.imread(race)
        try:
            img = tf.convert_to_tensor(img)
            img_ls.append(img)
        except:
            to_drop_ls.append('race/'+race.split('/')[-1].split('.')[0].replace('_', '/'))
        
    season_ls_img.append(np.array(img_ls))
    
season_ls_img = np.array(season_ls_img)
to_drop_ls = list(dict.fromkeys(to_drop_ls))
    

[ WARN:0@36.325] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/nc-poland-itt_2020_result.jpg'): can't open/read file: check file path/integrity
[ WARN:0@36.325] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/nc-poland_2020_result.jpg'): can't open/read file: check file path/integrity
[ WARN:0@36.339] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/nc-czech-republic-itt_2020_result.jpg'): can't open/read file: check file path/integrity
[ WARN:0@36.339] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/nc-czech-republic_2020_result.jpg'): 

In [7]:
result_ls_2 = [list(itertools.chain(*x[~x.race_ref.isin(to_drop_ls)].drop(columns='race_ref').values)) for x in result_ls]

In [36]:
def get_sequences_img(season_ls, result_ls):
    new_season_ls = []
    new_result_ls = []
    
    for season, results in zip(season_ls, result_ls):
        if season.shape[0] == 0:
            pass
        else:
            tensors_season = tf.convert_to_tensor(season, dtype=tf.float32)
            new_season_ls.append(tensors_season)
            
            tensors_y = tf.convert_to_tensor(results, dtype=tf.float32)
            new_result_ls.append(tensors_y)
        
    X_encoder_train = tf.ragged.stack(new_season_ls).to_tensor()
    y_encoder_train = tf.ragged.stack(new_result_ls).to_tensor()
    
    return X_encoder_train, y_encoder_train
        

In [38]:
X_encoder_train, y_encoder_train = get_sequences_img(season_ls_img, result_ls_2)

X_encoder_train.shape, y_encoder_train.shape

(TensorShape([394, 40, 150, 300, 3]), TensorShape([394, 40]))